In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = 9, 6
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%run survey_lcs.py 64 2 0.25 --gpu_frac 0.6 --gpu_id 1

Using TensorFlow backend.


[]
/Users/brettnaul/Dropbox/Documents/timeflow/keras_logs/survey_lcs/gru_064_x2_1m03_drop25
Loading /Users/brettnaul/Dropbox/Documents/timeflow/keras_logs/survey_lcs/gru_064_x2_1m03_drop25/weights.h5...
